In [ ]:
import json
from tqdm.notebook import tqdm

import torch
from datasets import load_from_disk
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

from peft import (
    PeftModel,
    PeftConfig,
)

# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

## Inference AFTER LoRA tuning

## Load data

In [ ]:
SEED = 42

# Load preprocessed dataset
dataset = load_from_disk("data/processed_data_no_removal").shuffle(seed=SEED)

# Dictionary to switch labels and IDs
label2id = {l: id for id, l in enumerate(sorted(set(dataset['train']['label'])))}
id2label = {id: l for l, id in label2id.items()}

## Load model

In [ ]:
# Directory of LoRA model
dir_model = './lora_result_25tweets_lr-1e-4/after-5epoch'

# Load the configuration for LoRA model
config = PeftConfig.from_pretrained(dir_model)
inference_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.bfloat16,
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, dir_model)

# Merge the adapter weights with the base model which allows the model's inference to speed up
merged_model = inference_model.merge_and_unload()

## Prediction process

In [ ]:
cor_a = 0
tot_a = 0
pred_list_a = []

for dp in tqdm(dataset['test']):
    inputs = tokenizer(dp['text'], return_tensors="pt")

    with torch.no_grad():
        outputs = merged_model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).item()
        predictions, references = merged_model.config.id2label[predictions], dp['label']

    tot_a += 1
    if predictions == references:
        cor_a += 1

    pred_list_a.append(predictions)

accuracy_a = cor_a / tot_a
print(f"Accuracy after LoRA: {accuracy_a}")

## Save outputs

In [ ]:
results_a = {
    'text': dataset['test']['text'],
    'references': dataset['test']['label'],
    'predictions': pred_list_a
}

with open("./lora_result_25tweets_lr-1e-4/inference_after_lora_5epoch.json", "w") as f:
    json.dump(results_a, f)

## Evaluate results

In [ ]:
def print_evaluation_result(references, predictions):
    cor = [0] * 4
    total_cor = 0

    for ref, pred in tqdm(zip(refs, preds)):
        # Compare elements and increment the corresponding count in cor if they match
        for i, (r_trait, p_trait) in enumerate(zip(ref, pred)):
            if r_trait == p_trait:
                cor[i] += 1

        # For multi-class classification
        if ref == pred:
            total_cor += 1

    accuracy = [round(c / len(refs), 4) for c in cor]
    total_accuracy = round(total_cor / len(refs), 4)

    print(f"Accuracy for each dimension: {accuracy}")
    print(f"Accuracy for multi-class classification: {total_accuracy}")



with open('lora_result_25tweets_lr-1e-4/inference_after_lora_1700steps.json', 'r') as before_f:
    before_result = json.load(before_f)
    
refs, preds = before_result['references'], before_result['predictions']
print_evaluation_result(refs, preds)